In [24]:
spark.stop()

In [25]:
from pyspark.sql import SparkSession
import getpass

username = getpass.getuser()

spark = SparkSession.builder \
    .appName("itv023333-shubham") \
    .master("yarn") \
    .config("spark.ui.port", "0") \
    .config("spark.sql.warehouse.dir", f"/user/{username}/warehouse") \
    .enableHiveSupport() \
    .getOrCreate()


In [26]:
spark

In [27]:
orders_schema = "customer_id long, order_date string, order_id long, order_status string"

In [28]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [29]:
orders_df.show(5)

+-----------+--------------------+--------+---------------+
|customer_id|          order_date|order_id|   order_status|
+-----------+--------------------+--------+---------------+
|          1|2013-07-25 00:00:...|   11599|         CLOSED|
|          2|2013-07-25 00:00:...|     256|PENDING_PAYMENT|
|          3|2013-07-25 00:00:...|   12111|       COMPLETE|
|          4|2013-07-25 00:00:...|    8827|         CLOSED|
|          5|2013-07-25 00:00:...|   11318|       COMPLETE|
+-----------+--------------------+--------+---------------+
only showing top 5 rows



In [30]:
customer_schema = "customer_id long, fname string, lname string, username string, password string, address string, city string, state string, pincode long"

In [31]:
customer_df = spark.read \
.format("csv") \
.schema(customer_schema) \
.load("/public/trendytech/retail_db/customers")

In [32]:
customer_df.show(5)

+-----------+-------+---------+---------+---------+--------------------+-----------+-----+-------+
|customer_id|  fname|    lname| username| password|             address|       city|state|pincode|
+-----------+-------+---------+---------+---------+--------------------+-----------+-----+-------+
|          1|Richard|Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|Brownsville|   TX|  78521|
|          2|   Mary|  Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|  Littleton|   CO|  80126|
|          3|    Ann|    Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|     Caguas|   PR|    725|
|          4|   Mary|    Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common| San Marcos|   CA|  92069|
|          5| Robert|   Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|     Caguas|   PR|    725|
+-----------+-------+---------+---------+---------+--------------------+-----------+-----+-------+
only showing top 5 rows



In [40]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold',10485760)

In [41]:
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'10485760'

broadcast join

In [42]:
orders_df.join(customer_df,orders_df.customer_id == customer_df.customer_id, "inner").write.format("noop").mode("overwrite").save()

In [46]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold',-1)

In [47]:
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'-1'

shuffle sort merge join (normal join)

In [ ]:
orders_df.join(customer_df,orders_df.customer_id == customer_df.customer_id, "inner").write.format("noop").mode("overwrite").save()

shuffle hash join using hint

In [ ]:
orders_df.join(customer_df.hint("shuffle_hash"),orders_df.customer_id == customer_df.customer_id, "inner").write.format("noop").mode("overwrite").save()

Joins using AQE (ADAPTIVE QUERY EXECUTION)

In [43]:
spark.conf.get("spark.sql.adaptive.enabled")


'false'

In [44]:
spark.conf.set("spark.sql.adaptive.enabled",True)

In [45]:
spark.conf.get("spark.sql.adaptive.enabled")

'true'

In [49]:
orders_df.join(customer_df,orders_df.customer_id == customer_df.customer_id, "inner").write.format("noop").mode("overwrite").save()

Py4JJavaError: An error occurred while calling o398.save.
: org.apache.spark.SparkException: Writing job aborted.
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:388)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:336)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExec.writeWithV2(WriteToDataSourceV2Exec.scala:241)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExec.run(WriteToDataSourceV2Exec.scala:255)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:40)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:40)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.doExecute(V2CommandExec.scala:55)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.

In [50]:
left_data = [
    ("101",1,"closed"),
    ("102",2,"complete"),
    ("103",3,"pending"),
    ("104",4,"closed")
]

left_df= spark.createDataFrame(left_data,["order_id", "cust_id","status string"])

left_df.show()

+--------+-------+-------------+
|order_id|cust_id|status string|
+--------+-------+-------------+
|     101|      1|       closed|
|     102|      2|     complete|
|     103|      3|      pending|
|     104|      4|       closed|
+--------+-------+-------------+



In [51]:
right_data = [
    (1,"banglore"),
    (2,"pune"),
    (5,"mumbai")
]

right_df = spark.createDataFrame(right_data, ["cust_id","city"])

right_df.show()

+-------+--------+
|cust_id|    city|
+-------+--------+
|      1|banglore|
|      2|    pune|
|      5|  mumbai|
+-------+--------+



semi join will have matching records but only returns from left table (which can match to right table )

In [52]:
left_df.join(right_df, left_df.cust_id == right_df.cust_id,"semi").show()

+--------+-------+-------------+
|order_id|cust_id|status string|
+--------+-------+-------------+
|     101|      1|       closed|
|     102|      2|     complete|
+--------+-------+-------------+



anti or left anti join is opposite of semi join it returns non matching records but only from left table 

In [54]:
left_df.join(right_df, left_df.cust_id == right_df.cust_id,"anti").show()

+--------+-------+-------------+
|order_id|cust_id|status string|
+--------+-------+-------------+
|     103|      3|      pending|
|     104|      4|       closed|
+--------+-------+-------------+

